# Advanced Station Assignment
Each Landkreis is now assigned _three_ weather stations. Their measurements are weighted according to their distance to the Landkreis center.

In [21]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import math

In [22]:
path_base = Path.cwd()

# export path
path_export = Path.joinpath(path_base, "exports")
path_export.mkdir(parents=True, exist_ok=True)

In [3]:
# import the data from Notebook 01
df_temp = pd.read_pickle(Path.joinpath(path_export, "temp.pkl"))
df_temp_stations = pd.read_pickle(Path.joinpath(path_export, "temp_stations.pkl"))

df_prec = pd.read_pickle(Path.joinpath(path_export, "prec.pkl"))
df_prec_stations = pd.read_pickle(Path.joinpath(path_export, "prec_stations.pkl"))

df_sun = pd.read_pickle(Path.joinpath(path_export, "sun.pkl"))
df_sun_stations = pd.read_pickle(Path.joinpath(path_export, "sun_stations.pkl"))

df_wind = pd.read_pickle(Path.joinpath(path_export, "wind.pkl"))
df_wind_stations = pd.read_pickle(Path.joinpath(path_export, "wind_stations.pkl"))

## Import Geospatial Data

In [4]:
# load RKI Covid-19 data in order to build a Landkreis-ID lookup table
df_rki = pd.read_csv("https://www.arcgis.com/sharing/rest/content/items/f10774f1c63e40168479a1feb6c7ca74/data")
df_landkreise = df_rki.drop_duplicates('Landkreis')[['Landkreis', 'IdLandkreis', 'Bundesland', 'IdBundesland']]
df_landkreise

,Landkreis,IdLandkreis,Bundesland,IdBundesland
0,SK Flensburg,1001,Schleswig-Holstein,1
34,SK Kiel,1002,Schleswig-Holstein,1
297,SK Lübeck,1003,Schleswig-Holstein,1
449,SK Neumünster,1004,Schleswig-Holstein,1
524,LK Dithmarschen,1051,Schleswig-Holstein,1
...,...,...,...,...
137473,LK Saalfeld-Rudolstadt,16073,Thüringen,16
137539,LK Saale-Holzland-Kreis,16074,Thüringen,16
137599,LK Saale-Orla-Kreis,16075,Thüringen,16
137725,LK Greiz,16076,Thüringen,16


In [5]:
# load geospatial data of the Landkreise in Germany
df_districts_geo = pd.read_csv("https://public.opendatasoft.com/explore/dataset/landkreise-in-germany/download/?format=csv&timezone=Europe/Berlin&lang=en&use_labels_for_header=true&csv_separator=%3B", ";")
df_districts_geo
# Our districtId is in column "Cca 2"

,Geo Point,Geo Shape,Id 0,ISO,Name 0,Id 1,Name 1,Id 2,Name 2,Hasc 2,Ccn 2,Cca 2,Type 2,Engtype 2,Nl Name 2,Varname 2
0,"47.9925229956,7.81807596197","{""type"": ""Polygon"", ""coordinates"": [[[7.790447...",86,DEU,Germany,1,Baden-Württemberg,12,Freiburg im Breisgau,DE.BW.FB,0,8311.0,Stadtkreis,District,NaN,NaN
1,"48.5964037974,10.527764168","{""type"": ""Polygon"", ""coordinates"": [[[10.61448...",86,DEU,Germany,2,Bayern,68,Dillingen an der Donau,DE.BY.DD,0,9773.0,Landkreis,District,NaN,NaN
2,"49.4362114486,11.0827553426","{""type"": ""MultiPolygon"", ""coordinates"": [[[[11...",86,DEU,Germany,2,Bayern,107,Nürnberg,DE.BY.NR,0,9564.0,Kreisfreie Stadt,District,NaN,NaN
3,"49.2159614099,11.5665579197","{""type"": ""Polygon"", ""coordinates"": [[[11.46063...",86,DEU,Germany,2,Bayern,110,Neumarkt in der Oberpfalz,DE.BY.NO,0,9373.0,Landkreis,District,NaN,NaN
4,"47.8443777181,12.1087247511","{""type"": ""Polygon"", ""coordinates"": [[[12.05431...",86,DEU,Germany,2,Bayern,122,Rosenheim,DE.BY.RH,0,9163.0,Kreisfreie Stadt,District,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,"52.6800481605,11.2270452746","{""type"": ""Polygon"", ""coordinates"": [[[11.51008...",86,DEU,Germany,13,Sachsen-Anhalt,339,Altmarkkreis Salzwedel,DE.ST.AS,0,15081.0,Landkreis,District,NaN,NaN
399,"51.7954450255,12.1432020671","{""type"": ""MultiPolygon"", ""coordinates"": [[[[11...",86,DEU,Germany,13,Sachsen-Anhalt,340,Anhalt-Bitterfeld,DE.ST.AB,0,15082.0,Landkreis,District,NaN,NaN
400,"52.2207122989,11.3478384337","{""type"": ""Polygon"", ""coordinates"": [[[11.06190...",86,DEU,Germany,13,Sachsen-Anhalt,341,Börde,DE.ST.BR,0,15083.0,Landkreis,District,NaN,NaN
401,"50.9558166403,13.1375068135","{""type"": ""Polygon"", ""coordinates"": [[[13.18351...",86,DEU,Germany,14,Sachsen,361,Mittelsachsen,DE.SN.MT,0,14522.0,Landkreis,District,NaN,NaN


In [6]:
df_lk = pd.merge(df_landkreise, df_districts_geo, left_on="IdLandkreis", right_on="Cca 2")[['Landkreis', 'Name 2', 'Type 2','IdLandkreis', 'Bundesland', 'Geo Point']]
df_lk = df_lk.rename(columns={'Name 2': 'Name kurz', 'Type 2': 'Typ'})
df_lk

,Landkreis,Name kurz,Typ,IdLandkreis,Bundesland,Geo Point
0,SK Flensburg,Flensburg,Kreisfreie Stadt,1001,Schleswig-Holstein,"54.7849933768,9.43852835486"
1,SK Kiel,Kiel,Kreisfreie Stadt,1002,Schleswig-Holstein,"54.3248406926,10.1322443646"
2,SK Lübeck,Lübeck,Kreisfreie Stadt,1003,Schleswig-Holstein,"53.8723167338,10.7272831058"
3,SK Neumünster,Neumünster,Kreisfreie Stadt,1004,Schleswig-Holstein,"54.0811244365,9.98448195474"
4,LK Dithmarschen,Dithmarschen,Kreis,1051,Schleswig-Holstein,"54.1329109614,9.10781447873"
...,...,...,...,...,...,...
394,LK Saalfeld-Rudolstadt,Saalfeld-Rudolstadt,Landkreis,16073,Thüringen,"50.637797959,11.3091162493"
395,LK Saale-Holzland-Kreis,Saale-Holzland-Kreis,Landkreis,16074,Thüringen,"50.904172137,11.7315307817"
396,LK Saale-Orla-Kreis,Saale-Orla-Kreis,Landkreis,16075,Thüringen,"50.5808480206,11.7105737336"
397,LK Greiz,Greiz,Landkreis,16076,Thüringen,"50.7484595538,12.0740705739"


In [7]:
# split up column "Geo Point" into two seperate numerical columns
df_lk['latitude'], df_lk['longitude'] = df_lk['Geo Point'].str.split(',', 1).str
df_lk[['latitude', 'longitude']] = df_lk[['latitude', 'longitude']].apply(pd.to_numeric)
df_lk.drop(columns=['Geo Point'], inplace=True)
df_lk

/Users/fynnwithake/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  


,Landkreis,Name kurz,Typ,IdLandkreis,Bundesland,latitude,longitude
0,SK Flensburg,Flensburg,Kreisfreie Stadt,1001,Schleswig-Holstein,54.784993,9.438528
1,SK Kiel,Kiel,Kreisfreie Stadt,1002,Schleswig-Holstein,54.324841,10.132244
2,SK Lübeck,Lübeck,Kreisfreie Stadt,1003,Schleswig-Holstein,53.872317,10.727283
3,SK Neumünster,Neumünster,Kreisfreie Stadt,1004,Schleswig-Holstein,54.081124,9.984482
4,LK Dithmarschen,Dithmarschen,Kreis,1051,Schleswig-Holstein,54.132911,9.107814
...,...,...,...,...,...,...,...
394,LK Saalfeld-Rudolstadt,Saalfeld-Rudolstadt,Landkreis,16073,Thüringen,50.637798,11.309116
395,LK Saale-Holzland-Kreis,Saale-Holzland-Kreis,Landkreis,16074,Thüringen,50.904172,11.731531
396,LK Saale-Orla-Kreis,Saale-Orla-Kreis,Landkreis,16075,Thüringen,50.580848,11.710574
397,LK Greiz,Greiz,Landkreis,16076,Thüringen,50.748460,12.074071


### Landkreise that are not covered by this dataset
The RKI dataset gives data for 412 Landkreise, however, the dataset from _opendatasoft_ provides geospatial coordinates only for 399 of them.

The Landkreise for which no geospatial data exists will be neglected in the following. If we have a look at them, we see that its mostly the districts of Berlin that are special, so we treat Berlin as a whole in the future:

In [8]:
# some of the Landkreise are not covered by BOTH datasets, so they will be omitted
pd.concat([df_lk, df_landkreise]).drop_duplicates(['IdLandkreis'], keep=False)

,Landkreis,Name kurz,Typ,IdLandkreis,Bundesland,latitude,longitude,IdBundesland
6877,LK Göttingen,NaN,NaN,3159,Niedersachsen,NaN,NaN,3.0
120750,SK Berlin Mitte,NaN,NaN,11001,Berlin,NaN,NaN,11.0
121534,SK Berlin Friedrichshain-Kreuzberg,NaN,NaN,11002,Berlin,NaN,NaN,11.0
121979,SK Berlin Pankow,NaN,NaN,11003,Berlin,NaN,NaN,11.0
122573,SK Berlin Charlottenburg-Wilmersdorf,NaN,NaN,11004,Berlin,NaN,NaN,11.0
123221,SK Berlin Spandau,NaN,NaN,11005,Berlin,NaN,NaN,11.0
123485,SK Berlin Steglitz-Zehlendorf,NaN,NaN,11006,Berlin,NaN,NaN,11.0
123972,SK Berlin Tempelhof-Schöneberg,NaN,NaN,11007,Berlin,NaN,NaN,11.0
124571,SK Berlin Neukölln,NaN,NaN,11008,Berlin,NaN,NaN,11.0
125220,SK Berlin Treptow-Köpenick,NaN,NaN,11009,Berlin,NaN,NaN,11.0


### Add Göttingen
As _Geo Point_ I take the coordinates of the city of Göttingen.

In [9]:
df_lk = pd.concat([df_lk, df_landkreise[df_landkreise['IdLandkreis'] == 3159]])
df_lk.set_index('IdLandkreis', inplace=True)
df_lk.loc[3159, 'Name kurz'] = "Göttingen"
df_lk.loc[3159, 'Typ'] = "Landkreis"
df_lk.loc[3159, 'latitude'] = 51.540120
df_lk.loc[3159, 'longitude'] = 9.930627
df_lk.reset_index(inplace=True)
df_lk

,IdLandkreis,Landkreis,Name kurz,Typ,Bundesland,latitude,longitude,IdBundesland
0,1001,SK Flensburg,Flensburg,Kreisfreie Stadt,Schleswig-Holstein,54.784993,9.438528,NaN
1,1002,SK Kiel,Kiel,Kreisfreie Stadt,Schleswig-Holstein,54.324841,10.132244,NaN
2,1003,SK Lübeck,Lübeck,Kreisfreie Stadt,Schleswig-Holstein,53.872317,10.727283,NaN
3,1004,SK Neumünster,Neumünster,Kreisfreie Stadt,Schleswig-Holstein,54.081124,9.984482,NaN
4,1051,LK Dithmarschen,Dithmarschen,Kreis,Schleswig-Holstein,54.132911,9.107814,NaN
...,...,...,...,...,...,...,...,...
395,16074,LK Saale-Holzland-Kreis,Saale-Holzland-Kreis,Landkreis,Thüringen,50.904172,11.731531,NaN
396,16075,LK Saale-Orla-Kreis,Saale-Orla-Kreis,Landkreis,Thüringen,50.580848,11.710574,NaN
397,16076,LK Greiz,Greiz,Landkreis,Thüringen,50.748460,12.074071,NaN
398,16077,LK Altenburger Land,Altenburger Land,Landkreis,Thüringen,50.956425,12.399131,NaN


## Aggregate Weather Data by Day
Eventually, we want to have weather parameters for each day for each Landkreis. So far, the measurements are on an hourly resolution. I take the daily mean of the temperatures, and the sum of the precipitatino and sunshine hour data per day. 

In [10]:
temp = df_temp.groupby(['station_id', pd.Grouper(key='date', freq='D')]).mean().reset_index()
prec = df_prec.groupby(['station_id', pd.Grouper(key='date', freq='D')]).sum().reset_index()
sun = df_sun.groupby(['station_id', pd.Grouper(key='date', freq='D')]).sum().reset_index()
wind = df_wind.groupby(['station_id', pd.Grouper(key='date', freq='D')]).mean().reset_index()

# prettify
temp.drop(columns=['quality'], inplace=True)

prec.drop(columns=['quality', 'R1_IND', 'WRTR'], inplace=True)
prec.rename(columns={'R1': 'precipitation'}, inplace=True)

sun.drop(columns=['quality'], inplace=True)
sun.rename(columns={'SD_SO': 'sunshine'}, inplace=True)

wind.drop(columns=['quality'], inplace=True)

In [11]:
wind

,station_id,date,velocity,direction
0,11,2020-01-01,1.383333,189.583333
1,11,2020-01-02,2.762500,172.083333
2,11,2020-01-03,3.091667,172.500000
3,11,2020-01-04,3.008333,262.500000
4,11,2020-01-05,1.537500,233.333333
...,...,...,...,...
38487,15978,2020-05-13,2.666667,81.666667
38488,15978,2020-05-14,4.433333,90.416667
38489,15978,2020-05-15,3.520833,92.500000
38490,15978,2020-05-16,1.862500,129.583333


## Landkreis–Station Matching
This matching algorithm assigns each Landkreis _three_ different weather stations. The measurements of these stations will be weighted according to their proximity to the respective Landkreis center.
For now, I use this simple formula for weighting point $a$ relative to points $b$ and  $c$:
$$W(a) = \frac{a^{-1}}{a^{-1} + b^{-1} + c^{-1}}\qquad \in (0, 1), \; W(a) + W(b) + W(c) = 1$$


In [12]:
def assign_weather_station_to_landkreis(df_stations, df_lk, df_weather):
    """Compares the center of each Landkreis with the location of each weather station
       and finds the THREE stations that are closest to a particular Landkreis center.
    """
    
    # filter out stations that don't provide data in df_weather
    not_allowed = pd.concat([df_stations, df_weather]).drop_duplicates('station_id', keep=False)
    df_stations = df_stations[~df_stations['station_id'].isin(not_allowed['station_id'])]
    
    closest_stations_dict = {}
    for lk_idx, lk_row in tqdm(df_lk.iterrows(), total=df_lk.shape[0]):
        idLandkreis = lk_row['IdLandkreis']
        for idx, row in df_stations.iterrows():
            
            # calculate distance between station and landkreis center
            lk_lat = lk_row['latitude']
            lk_lon = lk_row['longitude']

            station_lat = row['latitude']
            station_lon = row['longitude']

            a = station_lat - lk_lat
            b = station_lon - lk_lon
            distance = math.sqrt(a*a + b*b)

            if idLandkreis not in closest_stations_dict.keys():
                closest_stations_dict[idLandkreis] = [{'station_id': row['station_id'], 'distance': distance}]
            elif len(closest_stations_dict[idLandkreis]) < 3:
                closest_stations_dict[idLandkreis].append({'station_id': row['station_id'], 'distance': distance})
            else:
                # find max distance
                max_distance = -1
                idx = None
                for i, station in enumerate(closest_stations_dict[idLandkreis]):
                    if station['distance'] > max_distance:
                        max_distance = station['distance']
                        idx = i
                
                # check if current station is closer to landkreis
                if distance < max_distance:
                    closest_stations_dict[idLandkreis].append({'station_id': row['station_id'], 'distance': distance})
                    # remove old entry with larger max_distance
                    closest_stations_dict[idLandkreis].pop(idx)

        # now calculate weights for this Landkreis
        a = closest_stations_dict[idLandkreis][0]['distance']
        b = closest_stations_dict[idLandkreis][1]['distance']
        c = closest_stations_dict[idLandkreis][2]['distance']
        
        a, b, c = weight_three_points(a, b, c)
        
        closest_stations_dict[idLandkreis][0]['weight'] = a
        closest_stations_dict[idLandkreis][1]['weight'] = b
        closest_stations_dict[idLandkreis][2]['weight'] = c
                    
    df = pd.DataFrame.from_dict(closest_stations_dict, orient='index').reset_index()
    df.rename(columns={'index': "IdLandkreis"}, inplace=True)
    return df

def weight_three_points(a, b, c):
    x = (1/a) / (1/a + 1/b + 1/c)
    y = (1/b) / (1/a + 1/b + 1/c)
    z = (1/c) / (1/a + 1/b + 1/c)
    
    return (x, y, z)

In [13]:
temp_lk_stations = assign_weather_station_to_landkreis(df_temp_stations, df_lk, df_temp)
prec_lk_stations = assign_weather_station_to_landkreis(df_prec_stations, df_lk, df_prec)
sun_lk_stations = assign_weather_station_to_landkreis(df_sun_stations, df_lk, df_sun)
wind_lk_stations = assign_weather_station_to_landkreis(df_wind_stations, df_lk, df_wind)
wind_lk_stations

100%|██████████| 400/400 [00:26<00:00, 15.09it/s]


,IdLandkreis,0,1,2
0,1001,"{'station_id': 1379, 'distance': 0.06432745050...","{'station_id': 2303, 'distance': 0.48100361788...","{'station_id': 4466, 'distance': 0.28007254504..."
1,1002,"{'station_id': 2564, 'distance': 0.05372784611...","{'station_id': 2961, 'distance': 0.22483441086...","{'station_id': 6163, 'distance': 0.27142205625..."
2,1003,"{'station_id': 3086, 'distance': 0.07536562222...","{'station_id': 3897, 'distance': 0.26379311279...","{'station_id': 5078, 'distance': 0.18504659470..."
3,1004,"{'station_id': 2564, 'distance': 0.33591050708...","{'station_id': 4039, 'distance': 0.36406697275...","{'station_id': 6163, 'distance': 0.37695940177..."
4,1051,"{'station_id': 788, 'distance': 0.249785588996...","{'station_id': 1200, 'distance': 0.11636986966...","{'station_id': 2303, 'distance': 0.46790155000..."
...,...,...,...,...
395,16074,"{'station_id': 3821, 'distance': 0.26946044991...","{'station_id': 4464, 'distance': 0.34401343224...","{'station_id': 13711, 'distance': 0.3102354555..."
396,16075,"{'station_id': 2261, 'distance': 0.31554825913...","{'station_id': 4464, 'distance': 0.09441829136...","{'station_id': 13711, 'distance': 0.2201835784..."
397,16076,"{'station_id': 1612, 'distance': 0.14371099510...","{'station_id': 3946, 'distance': 0.27236390441...","{'station_id': 4464, 'distance': 0.32478587290..."
398,16077,"{'station_id': 1612, 'distance': 0.28047939872...","{'station_id': 2928, 'distance': 0.36175054534...","{'station_id': 5797, 'distance': 0.27073886118..."


Now we iterate over all Landkreise and assemble their weather data using the previously assigned weights

In [14]:
def derive_lk_weather(station_assignment, weather, which):
    # assign column names, depending on data type
    cols = []
    if which == "temp":
        cols = ['temperature', 'humidity']
    elif which == "prec":
        cols = ['precipitation']
    elif which == "sun":
        cols = ['sunshine']
    elif which == "wind":
        cols = ['velocity', 'direction']
    else:
        raise Exception(f"Unknown data type {which}")
    
    # iterate over Landkreise
    frames = []
    for i, lk_row in tqdm(station_assignment.iterrows(), total=station_assignment.shape[0]):
        
        # extract the three assigned stations and their corresponding weights
        stations_and_weights = []
        for j, item in lk_row.items():
            if j == "IdLandkreis":
                continue
            stations_and_weights.append(item)
        
        # extract and weight the weather data from each station
        dfs = []
        for s in stations_and_weights:
            df = weather[weather['station_id'] == s['station_id']].copy(deep=True)
            # weight each data column
            for c in cols:
                df[c] = df[c] * s['weight']
            
            df.drop(columns=['station_id'], inplace=True)
            dfs.append(df)
        # concatenate weighted data and sum values with same date
        df = pd.concat(dfs)
        
        # check if data is complete, i.e. three stations contributed for every measurement
        counts = df.groupby('date').count()
        bad_dates = counts[counts[cols[0]] != 3].reset_index()['date']
        
        # the last step in making a weighted sum is summing
        df = df.groupby('date').sum().reset_index()
        df = df[~df['date'].isin(bad_dates)]
        
        df['IdLandkreis'] = lk_row['IdLandkreis']
        frames.append(df)

    return pd.concat(frames)

            
            
            
            

In [15]:
temp = derive_lk_weather(temp_lk_stations, temp, "temp")
prec = derive_lk_weather(prec_lk_stations, prec, "prec")
sun = derive_lk_weather(sun_lk_stations, sun, "sun")
wind = derive_lk_weather(wind_lk_stations, wind, "wind")

100%|██████████| 400/400 [00:07<00:00, 56.29it/s]


In [16]:
temp[temp['IdLandkreis'] == 3403].head(20)

,date,temperature,humidity,IdLandkreis
0,2020-01-01,-0.797040,99.076015,3403
1,2020-01-02,0.374753,97.970351,3403
2,2020-01-03,6.598090,92.029549,3403
3,2020-01-04,5.105789,86.642993,3403
4,2020-01-05,4.390459,92.229820,3403
5,2020-01-06,4.804349,87.061010,3403
6,2020-01-07,6.277059,88.565714,3403
7,2020-01-08,9.171171,92.106436,3403
8,2020-01-09,10.673931,94.637006,3403
9,2020-01-10,7.442227,88.706364,3403


In [17]:
prec

,date,precipitation,IdLandkreis
0,2020-01-01,0.000000,1001
1,2020-01-02,0.000000,1001
2,2020-01-03,6.805375,1001
3,2020-01-04,2.184852,1001
4,2020-01-05,0.343244,1001
...,...,...,...
133,2020-05-13,0.000000,3159
134,2020-05-14,0.000000,3159
135,2020-05-15,0.000000,3159
136,2020-05-16,0.000000,3159


## Add Additional Information

In [18]:
temp = temp.merge(df_landkreise[['Landkreis', 'IdLandkreis']], on='IdLandkreis')
prec = prec.merge(df_landkreise[['Landkreis', 'IdLandkreis']], on='IdLandkreis')
sun = sun.merge(df_landkreise[['Landkreis', 'IdLandkreis']], on='IdLandkreis')
wind = wind.merge(df_landkreise[['Landkreis', 'IdLandkreis']], on='IdLandkreis')

## Export

In [19]:
# weather data
temp.to_pickle(Path.joinpath(path_export, "04_temp_final.pkl"))
prec.to_pickle(Path.joinpath(path_export, "04_prec_final.pkl"))
sun.to_pickle(Path.joinpath(path_export, "04_sun_final.pkl"))
wind.to_pickle(Path.joinpath(path_export, "04_wind_final.pkl"))

# station assignment
temp_lk_stations.to_pickle(Path.joinpath(path_export, "04_temp_stations_assigned.pkl"))
prec_lk_stations.to_pickle(Path.joinpath(path_export, "04_prec_stations_assigned.pkl"))
sun_lk_stations.to_pickle(Path.joinpath(path_export, "04_sun_stations_assigned.pkl"))
wind_lk_stations.to_pickle(Path.joinpath(path_export, "04_wind_stations_assigned.pkl"))

In [20]:
temp_lk_stations

,IdLandkreis,0,1,2
0,1001,"{'station_id': 1666, 'distance': 0.07946901664...","{'station_id': 2437, 'distance': 0.33803292320...","{'station_id': 4466, 'distance': 0.28007254504..."
1,1002,"{'station_id': 2564, 'distance': 0.05372784611...","{'station_id': 2961, 'distance': 0.22483441086...","{'station_id': 6163, 'distance': 0.27142205625..."
2,1003,"{'station_id': 1736, 'distance': 0.30297657622...","{'station_id': 3086, 'distance': 0.07536562222...","{'station_id': 3897, 'distance': 0.26379311279..."
3,1004,"{'station_id': 5280, 'distance': 0.28959113969...","{'station_id': 6105, 'distance': 0.29825011290...","{'station_id': 7427, 'distance': 0.08580912754..."
4,1051,"{'station_id': 1200, 'distance': 0.11636986966...","{'station_id': 1266, 'distance': 0.26660189390...","{'station_id': 1451, 'distance': 0.33641029069..."
...,...,...,...,...
395,16074,"{'station_id': 2444, 'distance': 0.14999789519...","{'station_id': 3289, 'distance': 0.18366673245...","{'station_id': 7420, 'distance': 0.20125739192..."
396,16075,"{'station_id': 3034, 'distance': 0.15067181449...","{'station_id': 3289, 'distance': 0.16445434481...","{'station_id': 4464, 'distance': 0.09441829136..."
397,16076,"{'station_id': 1612, 'distance': 0.14371099510...","{'station_id': 3946, 'distance': 0.27236390441...","{'station_id': 7419, 'distance': 0.08747292549..."
398,16077,"{'station_id': 4997, 'distance': 0.06085142699...","{'station_id': 5750, 'distance': 0.26064747928...","{'station_id': 5797, 'distance': 0.27073886118..."
